# Gene analysis, Drugs - gget opentargets

In [27]:
import os
import anndata as ad
import pandas as pd
import gget
# !pip install gget

In [48]:
base_dir = "/home/jrich/Desktop/CART_prostate_sc"  # os.path.dirname(os.path.dirname(__file__))
run_id = "SAMN10988512_SRX5414523_SRR8615230"
adata_processed_path = "/home/jrich/Desktop/CART_prostate_sc/data/output/SAMN10988512_SRX5414523_SRR8615230/data/adata_processed.h5ad"
cosmic_reference_with_aa_mutations_path = "/home/jrich/Desktop/CART_prostate_sc/data/reference/COSMIC/GRCh37/CancerMutationCensus_AllData_Tsv_v100_GRCh37/CancerMutationCensus_AllData_v100_GRCh37_gget_mutate_updated.csv"
n = 3

data_folder = os.path.join(base_dir, 'data')
output_figure_dir = os.path.join(data_folder, 'output', run_id, 'figures', 'mutation_protein')
output_data_dir = os.path.join(data_folder, 'output', run_id, 'data')

os.makedirs(output_figure_dir, exist_ok=True)
os.makedirs(output_data_dir, exist_ok=True)

adata = ad.read_h5ad(adata_processed_path)
adata.var.head()

,header,transcript_count,seq_ID,gene_name,mutation_id,mut_ID,mutation,full_sequence,variant_type,start_nucleotide,end_nucleotide,sequence_length,start_nucleotide_reverse,end_nucleotide_reverse,distance_from_5_prime,distance_from_3_prime,distance_5_prime_bin,distance_3_prime_bin
0,ENST00000439458_UNC80_COSV104380982_58896742,63412.0,ENST00000439458,UNC80,COSV104380982_58896742,UNC80_COSV104380982_58896742,c.8141_8142insTTTTTTTTTTCAAGCAGAAGACGGCATACGAG...,ATGGTGAAGAGGAAGAGCTCCGAGGGCCAGGAGCAGGACGGCGGCC...,insertion,8141.0,8142.0,9777.0,1635.0,1636.0,8141.0,1635.0,8140.0,1630.0
1,ENST00000266503_ARNTL2_COSV104369715_20849862,25806.0,ENST00000266503,ARNTL2,COSV104369715_20849862,ARNTL2_COSV104369715_20849862,c.947_948insTTTTTTTCAAGCAGAAGACGGCATACGAGATGAT...,ATGGCGGCGGAAGAGGAGGCTGCGGCGGGAGGTAAAGTGTTGAGAG...,insertion,947.0,948.0,1911.0,963.0,964.0,947.0,963.0,940.0,960.0
2,ENST00000294725_KCNT2_COSV54078235_26025088,19448.0,ENST00000294725,KCNT2,COSV54078235_26025088,KCNT2_COSV54078235_26025088,c.549_550insACGGCGACCACCGAGAT,ATGGTTGATTTGGAGAGCGAAGTGCCCCCTCTGCCTCCCAGGTACA...,insertion,549.0,550.0,3408.0,2858.0,2859.0,549.0,2858.0,540.0,2850.0
3,ENST00000598320_AC068620.1_COSV51704654_100123354,12439.0,ENST00000598320,AC068620.1,COSV51704654_100123354,AC068620.1_COSV51704654_100123354,c.414G>A,ATGGAGAGTGGAGCAGGAAAGCACTGGACTGAGGAGGAGGTTAAAG...,substitution,414.0,414.0,450.0,36.0,36.0,414.0,36.0,410.0,30.0
4,ENST00000593357_AC137056.1_COSV104978830_98529064,6053.0,ENST00000593357,AC137056.1,COSV104978830_98529064,AC137056.1_COSV104978830_98529064,c.299_302dup,ATGCTTTCAGTTTCTATACTCCATCAATTTGCTTGTCTTGGCTTCT...,duplication,299.0,302.0,408.0,106.0,109.0,299.0,106.0,290.0,100.0


In [15]:
# # For simplicity, filter out all mutants with ambiguous identity
# adata = adata[:, adata.var[~adata.var['header'].str.contains(';', na=False)].index].copy()

## Open Targets

In [49]:
from scripts.map_enst_to_ensg import get_ensembl_gene_id

adata.uns['combined_genes'] = adata.uns['combined_genes'][:n].copy()

adata.uns['combined_genes']['ensembl_gene_name'] = adata.uns['combined_genes']['seq_ID'].apply(get_ensembl_gene_id)

In [50]:
for idx, gene in enumerate(adata.uns['combined_genes']['ensembl_gene_name']):
    # !gget opentargets $gene --csv --out $output_data_dir/$gene.csv -r drugs
    df = gget.opentargets(gene, resource = 'drugs')
    if not df.empty:
        df.to_csv(os.path.join(output_data_dir, f'{idx}_{gene}_opentargets.csv'))
        print(f"Saved {gene} to {output_data_dir}")
    else:
        print(f"No data for {gene}")

11:32:44 - INFO - Retrieved 0/0 known drugs. (Querying count, will fetch all results next.)
11:32:44 - INFO - Retrieved 0/0 known drugs.


No data for ENSG00000144406


11:32:44 - INFO - Retrieved 0/0 known drugs. (Querying count, will fetch all results next.)
11:32:45 - INFO - Retrieved 0/0 known drugs.


No data for ENSG00000029153


11:32:46 - INFO - Retrieved 0/0 known drugs. (Querying count, will fetch all results next.)
11:32:46 - INFO - Retrieved 0/0 known drugs.


No data for ENSG00000162687
